<a href="https://colab.research.google.com/github/juanlu29/juanlu29/blob/econofisica/econofisica_odes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Modelo de economia cerrada. 

Quiero ver la distribucion de equilibrio de riqueza sobre una red de individuos que intercambian riqueza consumiendo y ofreciendo parte de sus recursos/servicios. La riqueza esta modelada como una cantidad conservativa y es constante sobre el total del sistema. Tenemos un sistema dinamico y las conexiones entre individuos se dan al azar definiendo una matriz de adyaciencia. Se pueden proponer diversos modelos estocásticos para la manera en que los enlaces de cada individuo se conecten con los demas de esta sociedad hipotetica. La ecuacion diferencial para modelar este proceso es como sigue

$ \dot{\vec{r}} = -\mathbb{I}_{\vec{r}} \mathbf{M}\vec{\gamma} + \mathbb{I}_{\vec{\gamma}}\mathbf{M}\vec{r}$

La distribucion de individuos están caracterizados por sus riquezas $\vec{r}$, cuyos elementos son el valor de la riqueza $r_{i}$ para individuos representados por el indice $i$. Son valores definidos positivos y la riqueza total $R = \sum_{i}^{N}{r_{i}}$ es constante por la naturaleza del modelo.

$\mathbf{M}$ es la matriz de adyaciencia, y es simétrica. Sus elementos valen la unidad cuando sus indices $i,j$, representando individuos $i$ y $j$, están conectados mediante una relacion de intercambio.

Las componentes del vector $\vec{\gamma}$ son el valor del servicio/productos ofrecidos por el individuo cuyo índice del elemento represente. Asi un individuo $i$ consume servicios $\gamma_{j}$ proporcional a su propia riqueza $r_{i}$ y vende sus servicios $\gamma_{i}$ proporcionalmente a la riqueza $r_{j}$ de quien la consume, respectivamente. De esta manera nos aseguramos la conservacion total de la riqueza.

Para expresar matricialmente el modelo anterior se definen las matrices diagonales $\mathbb{I}_{\vec{r}}, \mathbb{I}_{\vec{\gamma}}$ cuyos elementos son las componentes de $\vec{r}$ y $\vec{\gamma}$ respectivamente.


In [160]:
# Librerias
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

In [157]:
class r_punto():
  ''''
  El valor de la derivada temporal de la riqueza expresada matricialmente en la introduccion de este google colab
  '''
  def __init__(self,n,gamma,riquezas,tiempo):
     self.N = n # N individuos. Conlleva una matriz de adyaciencia de N x N
     # El valor de cada individuo de la red introducido como un ndarray
     self.Riquezas = riquezas
     # La distribucion de riqueza en la red introducida como un ndarray
     self.Gamma = gamma
     # Comprobamos que la dimension de los anteriores valores sea n
     if gamma.size != n or riquezas.size != n:
       raise NameError("El numero de individuos nos se corresponde al de la distribucion de riquezas o valor de trabajo introducidos")
     # Calculamos la matriz de adyacencia, la derivada temporal del sistema y su jacobiano
     self.matrizAdyacencia_uniforme()
     self.matrizSistema()
     self.jacobianoSistema()
     self.t = tiempo
     


  def matrizAdyacencia_uniforme(self):
      '''
      Genera matriz de adyaciencia estableciendo una probabilidad uniforme p entre generar o no conexion
      '''
      distribucion_1D_np =  np.random.choice(2, size=int(((self.N*self.N-self.N)/2) + self.N))
      self.M = np.zeros((self.N,self.N))
      self.M[np.triu_indices(self.N, k = 0)] = distribucion_1D_np
      self.M = self.M + self.M.T - 2*np.diag(np.diag(self.M))


  def drdt(self,y,t):
    '''
    Devuelve la derivada temporal de las riquezas
    '''
    self.matrizSistema()
    return self.S_drdt

  def jacobiano(self,y,t):
    '''
    Devuelve jacobiano del sistema
    '''
    self.jacobianoSistema()
    return self.J_drdt

  def matrizSistema(self):
    '''
    Calcula la matriz que expresa la derivada temporal de la "riqueza"
    '''
    self.S_drdt = -np.matmul(np.matmul(np.diag(self.Riquezas),self.M),self.Gamma.T) + np.matmul(np.matmul(np.diag(self.Gamma),self.M),self.Riquezas.T)

  def jacobianoSistema(self):
    '''
    Calcula el Jacobiano de la matriz del sistema
    '''
    self.J_drdt = -np.matmul(np.eye(self.N,dtype=int),np.matmul(self.M,self.Gamma)) + np.matmul(np.diag(self.Gamma),self.M)

  def integracion(self,dt):
    '''
    Integra un paso temporal el sistema
    '''
    self.Riquezas = odeint(self.drdt, self.Riquezas, np.array([self.t,self.t+dt]), Dfun=self.jacobiano)[1]
    self.t = self.t + dt


In [182]:
n = 1000
x_rand = np.random.normal(loc=1., scale=10., size=n)
SS = 50.
riquezas = (1./np.sqrt(2*np.pi))*np.exp(-np.power(x_rand,2)/SS)
gamma = np.random.normal(loc=1., scale=1.0, size=n)
red_1 = r_punto(n,gamma,riquezas,0.)
y = []
t = []
tiempo = 0
dt = 0.00003
for i in range(1000):
  t.append(tiempo)
  y.append(red_1.Riquezas)
  red_1.integracion(dt)
  tiempo = tiempo + dt

y = np.array(y)
print(y[500])
for i in range(n):
  plt.plot(t,y[:,i])

KeyboardInterrupt: ignored